In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [3]:
%ls -ltrh /Users/laurentperrinet/quantic/grants/Polychronies/RSG-spikepatterns/Data/

total 2292904
-rw-r--r--  1 laurentperrinet  staff   211M Jun 14 10:42 H_Ready-Set_bin1ms_alignedReady.mat
-rw-r--r--  1 laurentperrinet  staff   407M Jun 14 10:55 G_Ready-Set_bin1ms_alignedReady.mat
-rw-r--r--@ 1 laurentperrinet  staff   2.0K Jun 15 12:29 README.md
-rw-r--r--@ 1 laurentperrinet  staff   183M Jun 27 15:52 G_Ready-Set_bin1ms_alignedReady_session=0.npz
-rw-r--r--@ 1 laurentperrinet  staff   212M Jun 27 15:52 G_Ready-Set_bin1ms_alignedReady_session=1.npz
-rw-r--r--@ 1 laurentperrinet  staff    17M Jun 27 15:52 H_Ready-Set_bin1ms_alignedReady_session=0.npz
-rw-r--r--@ 1 laurentperrinet  staff    68M Jun 27 15:52 H_Ready-Set_bin1ms_alignedReady_session=1.npz


In [4]:
i_session = 0
monkey = 'G'
path = Path.home()/'quantic'/'grants'/'polychronies'/'RSG-spikepatterns'/'Data'/f'{monkey}_Ready-Set_bin1ms_alignedReady_session={i_session}.npz'

dictdata = np.load(path)
data, id_neuron, id_short, t_s, t_p = dictdata['data'], dictdata['id_neuron'], dictdata['id_short'], dictdata['t_s'], dictdata['t_p']
data = data.astype(float)
data[data==255] = np.nan
N_time, N_neurons, N_trial = data.shape

N_time, N_neurons, N_trial

(1200, 107, 1498)

### computing the firing rates

In [35]:
t_s_max = int(t_s.max())
DeltaT = 100 # number of time bins on which we average
dt = 20 # temporal distance between points on which we compute the firing rate

timepoints = np.arange(DeltaT, t_s_max+1, dt)
N_timepoints = len(timepoints)
timepoints

In [51]:
fr = np.empty((N_timepoints, N_neurons, N_trial)) * np.nan
for i_timepoint, timepoint in enumerate(timepoints):
    fr[i_timepoint, :, :] = np.nansum(data[(timepoint-DeltaT):timepoint, :, :], axis=0) / (DeltaT/1000)

### logistic regression model with NaNs

### inference model

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader
torch.set_default_tensor_type("torch.DoubleTensor") # -> torch.tensor([1.2, 3]).dtype = torch.float64
# see https://sebastianraschka.com/faq/docs/pytorch-crossentropy.html#pytorch-loss-input-confusion-cheatsheet
criterion = torch.nn.BCELoss(reduction="mean") # loss divided by output size
#criterion = torch.nn.NLLLoss(reduction="mean") # loss divided by output size


class LogisticRegressionModel(torch.nn.Module):
    def __init__(self, N, n_classes, logit0=torch.log(1.*torch.ones(1)), logit_wt=torch.log(0.1*torch.ones(1))):
        super(LogisticRegressionModel, self).__init__() 
        # self.linear = torch.nn.Linear(N, n_classes, bias=bias)

        self.bias = torch.nn.Parameter(logit0 * torch.ones(1, n_classes))
        self.weight = torch.nn.Parameter(logit_wt * torch.ones((N_input, n_classes)))
        # self.nl = torch.nn.LogSoftmax(n_classes)
        self.nl = torch.nn.Sigmoid()

    def forward(self, X_input):
        # X_input.shape = N_batch, N_input
        mask = torch.isnan(X_input)
        X_input[mask] = 0.
        return self.nl(X_input @ self.weight +  self.bias)
        
    

In [ ]:
logistic_model = LogisticRegressionModel(N_input, n_classes)
X_input, p, y = get_data(W, seed, N_batch, p_NaN)
outputs = logistic_model(torch.tensor(X_input.astype(float)))
outputs

tensor([[2.8270e-11, 2.8270e-11, 2.8270e-11, 2.8270e-11, 2.8270e-11, 2.8270e-11,
         2.8270e-11, 2.8270e-11, 2.8270e-11, 2.8270e-11],
        [1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00],
        [7.4381e-01, 7.4381e-01, 7.4381e-01, 7.4381e-01, 7.4381e-01, 7.4381e-01,
         7.4381e-01, 7.4381e-01, 7.4381e-01, 7.4381e-01],
        [7.4768e-42, 7.4768e-42, 7.4768e-42, 7.4768e-42, 7.4768e-42, 7.4768e-42,
         7.4768e-42, 7.4768e-42, 7.4768e-42, 7.4768e-42]],
       grad_fn=<SigmoidBackward0>)

In [ ]:
learning_rate = 0.005
beta1, beta2 = 0.9, 0.999
betas = (beta1, beta2)
num_epochs = 2 ** 9 + 1
batch_size = 256
n_classes=10
amsgrad = True  # gives similar results
amsgrad = False # gives similar results

def fit_data(X_input, y, 
            learning_rate=learning_rate,
            batch_size=batch_size,  # gamma=gamma,
            num_epochs=num_epochs,
            betas=betas,
            verbose=False, **kwargs
        ):

    X, labels = torch.Tensor(X_input[:, None]), torch.Tensor(y[:, None])
    loader = DataLoader(
        TensorDataset(X, labels), batch_size=batch_size, shuffle=True
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    N_batch = X_input.shape[0]
    N = X_input.shape[1]
    n_classes = y.shape[1]
    logistic_model = LogisticRegressionModel(N, n_classes)
    # logistic_model = logistic_model.to(device)
    logistic_model.train()
    optimizer = torch.optim.Adam(
        logistic_model.parameters(), lr=learning_rate, betas=betas, amsgrad=amsgrad
    )
    for epoch in range(int(num_epochs)):
        logistic_model.train()
        losses = []
        for X_, labels_ in loader:
            # X_, labels_ = X_.to(device), labels_.to(device)
            outputs = logistic_model(X_)
            loss = criterion(outputs, labels_)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

        if verbose and (epoch % (num_epochs // 32) == 0):
            print(f"Iteration: {epoch} - Loss: {np.mean(losses):.5f}")

    logistic_model.eval()
    X, labels = torch.Tensor(X_input[:, None]), torch.Tensor(y[:, None])
    outputs = logistic_model(X)
    loss = criterion(outputs, labels).item()
    return logistic_model, loss


X_input, p, y = get_data(W, seed=seed, N_batch=100000, p_NaN=p_NaN)
logistic_model, loss = fit_data(X_input, y, verbose=True)
print("Final loss =", loss)

Iteration: 0 - Loss: 14.81964
Iteration: 16 - Loss: 0.09919
Iteration: 32 - Loss: 0.09770
Iteration: 48 - Loss: 0.09743
Iteration: 64 - Loss: 0.09731
Iteration: 80 - Loss: 0.09733
Iteration: 96 - Loss: 0.09737
Iteration: 112 - Loss: 0.09732
Iteration: 128 - Loss: 0.09733
Iteration: 144 - Loss: 0.09737
Iteration: 160 - Loss: 0.09730
Iteration: 176 - Loss: 0.09732
Iteration: 192 - Loss: 0.09733
Iteration: 208 - Loss: 0.09745
Iteration: 224 - Loss: 0.09733
Iteration: 240 - Loss: 0.09732
Iteration: 256 - Loss: 0.09734
Iteration: 272 - Loss: 0.09733
Iteration: 288 - Loss: 0.09740
Iteration: 304 - Loss: 0.09730
Iteration: 320 - Loss: 0.09742
Iteration: 336 - Loss: 0.09738
Iteration: 352 - Loss: 0.09738
Iteration: 368 - Loss: 0.09732
Iteration: 384 - Loss: 0.09736
Iteration: 400 - Loss: 0.09735
Iteration: 416 - Loss: 0.09736


In [ ]:
fig, ax = plt.subplots(figsize=(13, 13))
ax.plot([-3, 3], [-3, 3], 'r--')
#ax.plot(logistic_model.linear.bias.detach().numpy(), 'r')
# ax.plot(W[-1, :], 'r--')
ax.scatter(logistic_model.bias.detach().numpy(), W[-1, :])
ax.set_xlabel('input dimensions')
ax.set_ylabel('X_input')

In [ ]:
logistic_model.linear.weight.detach().numpy().shape, logistic_model.linear.weight.detach().numpy().ravel().shape, W[:-1, :].shape, W[:-1, :].ravel().shape